<a href="https://colab.research.google.com/github/blue-create/langlens/blob/main/to_publish/nationalities_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model zur Nennung von Nationalitäten
In diesem Notebook wird ein Bert-Model erstellt, welches erkennt ob in einem Text zu Partnerschaftsgewalt die Nationalitäten von Täter und Opfer genannt werden.

Schritte:

*   Vorbereiten der Daten
*   Modellieren
*   Evaluieren des Modelles




### Imports, Konstanten, Paths

In [ ]:
%%capture
!pip install transformers==4.20.0

In [ ]:
%%capture
!pip install datasets

In [ ]:
# Vorbereiten
from google.colab import drive
import tqdm as tqdm
import os
import json
import pandas as pd

# Modelling
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import numpy as np
from datasets import Dataset
import datasets

# Evaluation
import torch
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import  confusion_matrix, classification_report

In [ ]:
# Verbinden mit GDrive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/data/

In [ ]:
# Import des Skripts das die Annotierten Daten liest
from scripts import annotations

In [ ]:
#Definieren der Paths
folder_path = "filtered"

## Vorbereiten der Daten

### Einlesen von Annotationsdaten

In [ ]:
# Lesen und zusammenfügen aller annotierten Dateien
dfs={}
for doc in os.listdir("3_Annotated/new_ontology"):
  # Lesen der Json-Datei und als Pandas Dataframe speichern
  if doc.endswith(".json"):
    json_data=json.load(open("3_Annotated/new_ontology/"+doc, encoding="utf-8"))
    data=pd.DataFrame(json_data["documents"])
    data.loc[:,"file"]=doc
    dfs[doc]=data

# Mergen aller Files
data=pd.concat(dfs,ignore_index=True)
data=data[data.annotations.apply(len)!=0]

In [ ]:
# Extrahieren und Vorbereiten der annotierten Daten
data.loc[:,"artikel_id"]=data.attributes_flat.apply(lambda x: x["artikel_id"])
data.loc[:,"name"]=data.attributes_flat.apply(lambda x: x["name"])
data.loc[:,"titel"]=data.attributes_flat.apply(lambda x: x["titel"])
data.loc[:,"ressort"]=data.attributes_flat.apply(lambda x: x["ressort"])
data.loc[:,"annotations"]=data.annotations.apply(annotations.extract_annotations)
data.loc[:,"dice"]=data.annotations.apply(annotations.calculate_similarity,sim="dice")
data.loc[:,"annotations"]=data.apply(annotations.ground_truth_filter,min_coannotation=1,min_similarity=-1, similarity="dice",axis=1)

In [ ]:
# Einlesen des Datensetz mit Annotationen zur Nennung von Nationalitäten
nat=pd.read_csv("testing_nationalities_reviewed.csv")
nat=nat[~nat["real label"].isna()]
nat=nat.drop_duplicates("text")

# Auswählen eines zufälligen Datensatzes
not_nat=data.sample(nat.shape[0])
df=pd.concat([not_nat,nat]).sample(frac=1)

## Modelling



In [ ]:
# Umwandeln der Annotationskategorien in binär
# 0: keine Nennung von Nationalitäten
# 1: Nennung von Nationalitäten
df["label"]=(df["real label"]=="nationality").astype(int)
df=df[["text","artikel_id","name","label"]]

In [ ]:
# Vortrainierte Modelle, die getestet wurden

#model_id="nlptown/bert-base-multilingual-uncased-sentiment"
model_id="bert-base-german-cased"
#model_id="dbmdz/bert-base-german-cased"
#model_id="oliverguhr/german-sentiment-bert"
#model_id="krupper/text-complexity-classification"
#model_id="classla/xlm-roberta-base-multilingual-text-genre-classifier"
#model_id="deepset/bert-base-german-cased-hatespeech-GermEval18Coarse"

In [ ]:
# Laden des Vortrainieren Tokenizers
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# Erstellen der Vortrainierten Modells
model = AutoModelForSequenceClassification.from_pretrained(model_id,  num_labels=2,ignore_mismatched_sizes=True)

In [ ]:
# Teilen der Daten in Trainings und Testsets
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.6 * len(df)), int(.8 * len(df))])

In [ ]:
# Umwandeln der Daten in Dataset
ds=datasets.DatasetDict({"train":Dataset.from_dict(df_train),"val":Dataset.from_dict(df_val),"test":Dataset.from_dict(df_test)})

In [ ]:
#Definieren der Trainingsargumenten
args = TrainingArguments(
    f"national_filter",
    overwrite_output_dir=True,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    num_train_epochs=10,
    weight_decay=0.05,
   # metric_for_best_model="f1",
    load_best_model_at_end=True,
    #push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Funktion zum Tokenisieren und Formatieren des Input-Texts
def preprocess_data(df):
  return tokenizer(df["text"], padding=False)

In [ ]:
# Preprocessing/ Toeknizierung der Input-Texte
ds_encoded=ds.map(preprocess_data, batched=False)
ds_encoded.set_format("torch")

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

In [ ]:
# Verlustfunktion zur Evaluierung des Models
def compute_metrics(eval_pred):
    """ Funktion zum Evaluieren der Performance eines Text-Modells
    Parameters:
      - p (EvalPrediction): Vorhersagen des Textmodells
    Returns:
      - result (dict): Performance des Textmodells: Verlustfunktion, F1, Accuracy
    """
    logits, labels = eval_pred
    y_pred = np.argmax(logits, axis=-1)
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}

    return metrics

In [ ]:
 # Definieren der Modelparametern mit Verlustfunktionen
trainer = Trainer(
    model,
    args,
    train_dataset=ds_encoded["train"],
    eval_dataset=ds_encoded["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

In [ ]:
# Trainieren des Models
trainer.train()

## Model Evaluation
Vohersagen der Kategorien und Erstellen einer Konfusionsmatrix zur Evaluierung des Models

### Vorhersagen

In [ ]:
sigmoid = torch.nn.Sigmoid()

In [ ]:
# Zu testende Texte

test_pred=pd.DataFrame(ds["test"])

In [ ]:
# Funktion zur Vorhersage der Textkategorien
def predict_text(text):
  """ Funktion zur Vorhersage von Text-Kategorien
      - text (str): Text, dessen Kategorie vorhergesagt werden soll
    Returns:
      - predictions (list of int): Liste mit Integern mit Länge der Anzahl an Annotationskategorien, mit 1 Kategorie ist zutreffend und 0 nicht zutreffend
  """
  encoding = tokenizer(text, return_tensors="pt")
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
  outputs = trainer.model(**encoding)
  logits = outputs.logits
  probs = sigmoid(logits.squeeze().cpu())
  predictions = np.zeros(probs.shape)
  predictions[np.where(probs >= 0.5)] = 1
  return predictions

In [ ]:
# Vorhersage von Text-Kategorien
pred=[]
for t in ds["test"]["text"]:
  pred.append(predict_text(t))
test_pred["prediction"]=pred

### Evaluierung

In [ ]:
# Darstellen der Modelperformance in einer Konfusionsmatrix
print(confusion_matrix(test_pred.label, y_pred))
print(classification_report(test_pred.label, y_pred))